# Estimator design with state variable
The control law assumes that all the state variables are available for feed back. In most cases, not all the state variables are measured. The cost of the required sensors may be prohibitive, or it may be physically impossible to measure all of the state variables.
One method of estimating the state is to construct a full order model of the plant dynamics:

$$
\dot{\hat{\pmb x}} = \pmb F \pmb{\hat x} + \pmb G u
\label{eq:xhat} \tag{2.1}
$$

where $\pmb{\hat x}$ is the estimate of the actual state $\pmb x$. To study the dynamics of this estimator, we define the error in the estimate to be:

$$
\tilde{\pmb x} = \pmb x - \pmb{\hat x}
$$

Then the dynamics of this error system are given by:

$$
\dot{\tilde{\pmb x}} = \pmb F \tilde{\pmb x}, \space  \tilde{\pmb x}(0) - \hat{\pmb x}(0)
$$

The error convergences to zero for a stable system system (**F** stable), but we have no ability to influence the rate at which the state estimate converges to the true state. Furthermore, the error is converging to zero at the same rate as the natural dynamics of **F**. if this convergence rate were satisfactory, no control or estimation would be required. The golden rule is: when in trouble use feedback. Consider the feedback the difference between the measured and estimate outputs and correcting the model continuously with this error signal. The equation for this scheme is: 

$$
\dot{\hat{\pmb x}} = \pmb F \hat{\pmb x} + \pmb G u + \pmb L(y - \pmb H \hat{\pmb x})
\label{eq:dot_hat_x} \tag{2.2}
$$

Here **L** is a proportional gain defined as:
$$
\pmb L = [l_1,l_2,.....,l_n]^T
$$

and is chosen to achieve satisfactory error characteristics. The dynamics of the error can be obtained by subtracting the estimate $\eqref{eq:dot_hat_x}$ from $\eqref{eq:state_eq}$ to get the error equation:

$$
    \dot{\tilde{\pmb x}} = (\pmb F - \pmb LH)\tilde{\pmb x}
$$

and the characteristic equation of the error is now given by:

$$
    det[s \pmb I - (\pmb{F - LH})] = 0
    \label{eq:det_est} \tag{2.3}
$$

If we can choose $\pmb L$ so that $\pmb F - \pmb{LH}$ has stable and reasonably fast eigenvalues, then $\tilde{\pmb x}$ will decay to zero and remain there independent of the known forcing function $u(t)$ and its effect on the state $x(t)$ and irrespective on the initial condition $\tilde{\pmb x}(0)$. This means that $\hat{\pmb x(t)}$ will converge to $\pmb{x}(t)$, regardless of the value of $\hat{\pmb x}(0)$. However we can typically choose $\pmb L$ so that the error system is still at least stable and the error remains acceptably small, even with modeling errors and disturbing inputs. The selection of $\pmb L$ can be approached in exactly the same fashion as $\pmb K$ is selected in the control law design. If we specify the desired location of the estimator error poles as:

$$
    s_i = \beta_1, \beta_2,....\beta_n
$$

then the desired estimator characteristic equation is:

$$
    \alpha_e(s) = (s-\beta_1)(s-\beta_2)....(s-\beta_n)
    \label{eq:char_eq} \tag{2.4}
$$

We can then solve for $\pmb L$ by comparing coefficients in Eqs. $\eqref{eq:det_est}$ and $\eqref{eq:char_eq}$.

## The observability
As begun in the introduction it's time to spent some raw talking about the **observability** of state variables. In a development exactly parallel with the control-law case, we can find a transformation to take a given system to observer canonical form if and only if the system has structural properties that in this case we call as mention just like above. Roughly speaking, observability refers to our ability to deduce information about all the modes of the system by monitoring only the sensed outputs. Not observability results when some mode or subsystem is disconnected physically from the output and therefore no longer appears in the measurements. The mathematical test for determining observability is that the **observability matrix**:  

$$
\mathcal{O} = \begin{bmatrix}
\pmb H\\
\pmb{HF}\\
..\\
..\\
\pmb{HF}^{n-1}
\end{bmatrix}
\label{eq:obs_mat} \tag{2.5}
$$

must have independent columns. In general, we can find a transformation to observer canonical form if and only if the observability matrix is nonsingular (full rank).Note that is analogous to our earlier conclusion for transforming system matrices to control canonical form.
As with control law design, we could find the transformation to observer form, compute the gains from the equivalent of $\pmb F_o - \pmb L \pmb H_o$ and transform back. An alternative method of computing $\pmb L$ is to use the Ackermann's formula in estimator form: 

$$
\pmb L = \alpha_e(\pmb F)\mathcal{O}^{-1}
\begin{bmatrix}
0 \\
0 \\
.. \\
.. \\
1 \\
\end{bmatrix} 
$$

where $\mathcal{O}$ is the observability matrix given in $\eqref{eq:obs_mat}$.

### Hand-on labs of observability of the cart-pendulum
This code snippet goes to calculate the observability matrix when our measurement matrix **H** (in the code is C) select a sensor related to x cart position and then reply the calculation for $\theta$ measurement. It can be note the determinant is nonzero (full rank) in the first trial and zero in the second one. in particular  tuns out that in the second is impossible to reconstruct the x position because that coordinate does not affect the others; equations of cart pendulum are invariant for translations in x position. It's important reconstruct the cart position to have full estimate of state variable putting them in the LQR block for resulting compensator placed in the feed back (as it will see later).
```
clear all, close all, clc

m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = -1; % pendulum up (s=1)

A = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

B = [0; 1/M; 0; s*1/(M*L)];

C = [1 0 0 0]; 

obsv(A,C)
det(obsv(A,C))

%%  only observable if x measured... because x can't be
%% reconstructed
C = [0 0 1 0];

obsv(A,C)
det(obsv(A,C))
```
The terminal result:
```
ans =

    1.0000         0         0         0
         0    1.0000         0         0
         0   -0.2000    2.0000         0
         0    0.0400   -0.4000    2.0000


ans =

     4


ans =

         0         0    1.0000         0
         0         0         0    1.0000
         0    0.1000   -6.0000         0
         0   -0.0200    0.2000   -6.0000


ans =

     0
```

## Estimator pole selection. The optimal full-state estimation 
In comparison with the selection of controllers poles, estimator pole selection require us to be concerned with a much different relationship than with control effort. As in controller, there is a feedback term in the estimator that grows in magnitude as the requested speed of response increases. However this feedback is in the form of an electronic signal or a digital word in a computer, so its growth causes no special difficulty. The important consequence of increasing the speed of response of an estimator is that the bandwidth of the estimator becomes higher, thus causing more sensor noise to pass on to the control actuator. Like the controller the estimator design (and the poles selection) is a balance between good transient response and low-enough bandwidth that sensor noise does not significantly impair the actuator activity.   
When deriving the optimal full-state estimator, it is necessary to re-introduce disturbances to the state, $\pmb w_d$ , and sensor noise, $\pmb w_n$ :


\begin{array}{cc}
\dot{\pmb x} &= \pmb{Fx} + \pmb{Gu} + \pmb{w}_d\\
\pmb y &= \pmb Hx + \pmb Ju + \pmb{w}_n\\ 
\label{eq:augsys} \tag{2.6}
\end{array}


By hypothesis is assumed that disturbance and noise are zero-mean Gaussian processes with known covariances:

$$
\begin{array}{cc}
E(\pmb{w}_d(t) \pmb{w}_d(\tau)^*) &= Q \delta (t - \tau)\\
E(\pmb{w}_n(t) \pmb{w}_n(\tau)^*) &= R \delta (t - \tau)\\
\end{array}
$$

Here $E$ is the expected value and $\delta (.)$ is the Dirac delta function. The matrices $Q$ and $R$ are positive semi-definite with entries containing the covariances of the disturbance and noise terms. It is possible to obtain an estimate $\hat{\pmb x}$ of the full-state $\pmb x$ from measurements of the input $\pmb u$ and output $\pmb y$, via the following estimator dynamical system:

$$
\begin{array}{cc}
\dot{\hat{\pmb x}} &= \pmb F \hat{\pmb x} + \pmb Gu + \pmb L (\pmb y - \hat{\pmb y})\\
\hat{\pmb y} &= \pmb H \hat{\pmb x} + \pmb Ju\\
\end{array}
\label{eq:estimator}
$$

The matrices $\pmb F, \pmb G, \pmb H, \pmb J$ are obtained from the system model and the filter
gain $\pmb L$ f is determined via a similar procedure as in LQR. $\pmb L$ is given by:

$$
\pmb L = \pmb Y \pmb H^* \pmb R
$$

where $\pmb Y$ is the solution to another algebraic Riccati equation:

$$
    \pmb YF^* + \pmb FY - \pmb{YH}^* \pmb R^{-1} \pmb H \pmb Y + \pmb Q = \pmb 0
$$

This solution is commonly referred to as the Kalman filter, and it is the optimal full-state estimator with respect to the following cost function:

$$
    J = \lim\limits_{t \to \infty} {E[\tilde x(t)^* \tilde x(t)]}
$$

This cost function implicitly includes the effects of disturbance and noise, which are required to determine the optimal balance between aggressive estimation and noise attenuation. Thus, the Kalman filter is referred to as linear quadratic estimation (LQE), and has a dual formulation to the LQR optimization.
The estimator dynamical system is expressed in terms of the estimate $\hat{\pmb x}$ with inputs $\pmb y$ and $\pmb u$. If the system is observable it is possible to place the eigenvalues of $\pmb F − \pmb L \pmb H$ arbitrarily with choice of $\pmb L$ . When the eigenvalues of the estimator are stable, then the state estimate $\hat{\pmb x}$ converges to the full-state $\pmb x$ asymptotically, as long as the model faithfully captures the true system dynamics. To see this convergence, consider the dynamics of the estimation error $\pmb{\epsilon} = \pmb x − \pmb{\hat x}$:

$$
\begin{array}{cc}
\dot{\pmb{\epsilon}} = {\pmb{x}} - \hat{\pmb{x}}\\
                     = [\pmb{Fx} + \pmb{Gu} + \pmb{w}_d] - [(\pmb F - \pmb{LH}) \hat{\pmb x} + \pmb{Ly} + (\pmb G - \pmb{LJ})\pmb u]\\
                     = (\pmb F - \pmb{LH}) \epsilon + \pmb w_d - \pmb L \pmb w_n
\end{array}
$$

Therefore, the estimate $\hat{\pmb x}$ will converge to the true full state when $\pmb F − \pmb L \pmb H$ has stable eigenvalues. As with LQR, there is a trade-off between over-stabilization of these eigenvalues and the amplification of sensor noise. This is similar to the behavior of an inexperienced driver who may hold the steering wheel too tightly and will overreact to every minor bump and disturbance on the road.

### Plant disturbance and Noise measurement. Hand-on labs
To give a demostration of best choise of $L \equiv K_f$ it's considered a system with $J=0$ in $\eqref{eq:augsys}$ and pendulum in bottom position. It's better create a practical solution around a stable equilibrium point because the algebra is the same. The system represented in the [picture](#kfpic) shows the linearized system with input $u$ to be the force applied to the cart pendulum, $w_d$ the gaussian zero-mean disturbance applied to the model and the cart position ($\pmb H=[1 0 0 0]$) noised as output of state variable observed. The goal is reconstruct the full state estimate. To do it is necessary to re-represent the system like 4 *augmented* blocks to feed in Matlab functions:     

\begin{array}{cc}
\pmb Q &= 0.1 \mathbb{1} & \pmb x = \begin{bmatrix} x\\ \dot x\\ \theta\\      \dot{\theta} \end{bmatrix}\\
R &= 1   
\end{array}

\begin{equation} \label{eq:aug-eqs}
\begin{split}
\pmb{G}_A & = \begin{bmatrix}\pmb G & \pmb Q & \pmb{\underline{0}}x \pmb B\end{bmatrix}\\
\pmb{F}_A & = \pmb{F}\\
\pmb{H}_A & = \pmb H\\
\pmb{J}_A & = \begin{bmatrix} 0 & 0 & 0 & 0 & 0 & R \end{bmatrix}
\end{split}
\end{equation}

Add picture

According to the equations $\eqref{eq:estimator}$, by inspecting $\eqref{eq:state-form}$ and $\eqref{eq:output}$ the *new* estimator-system matrices are:

\begin{array}{cc}
\label{eq:kalman-system}
\pmb{F}_k &= \pmb F - \pmb{K}_f \pmb H\\
\pmb{G}_k &= \begin{bmatrix}\pmb B \pmb{K}_f \end{bmatrix}\\
\pmb{H}_k &= \mathbb{1}\\
\pmb{J}_k &= \mathbb{0}
\end{array}

The input $u$ is augmented also to taking into account of disturbance plant and noise measurement:

$$
\label{eq:u_noise}
\pmb{u}_{aug} = \begin{bmatrix} u & {Q}^2 * \pmb{u}_{dist} & u_{noise}\end{bmatrix}
$$

The code:
```
clear all, close all, clc

m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = -1; % pendulum up (s=1)

% y = [x; dx; theta; dtheta];
F = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

G = [0; 1/M; 0; s*1/(M*L)];

H = [1 0 0 0];  

J = zeros(size(H,1),size(G,2));

%%  Augment system with disturbances and noise
Q = .1*eye(4);  % disturbance covariance
R = 1;       % noise covariance

GA = [G Q 0*G];  % augment inputs to include disturbance and noise

sysH = ss(F,GA,H,[0 0 0 0 0 R]);  % build big state space system... with single output

sysFullOutput = ss(F,GA,eye(4),zeros(4,size(GA,2)));  % system with full state output, disturbance, no noise

%%  Build Kalman filter
[L,P,E] = lqe(F,Q,H,Q,R);  % design Kalman filter
Kf = (lqr(F',H',Q,R))';   % alternatively, possible to design using "LQR" code

sysKF = ss(F-L*H,[G L],eye(4),0*[G L]);  % Kalman filter estimator

%%  Estimate linearized system in "down" position (Gantry crane)
dt = .01;
t = dt:dt:50;

uDIST = randn(4,size(t,2));
uNOISE = randn(size(t));
u = 0*t;
u(100:120) = 100;     % impulse
u(1500:1520) = -100;  % impulse

uAUG = [u; Q*Q*uDIST; uNOISE];

[y,t] = lsim(sysC,uAUG,t);
[xtrue,t] = lsim(sysFullOutput,uAUG,t);


[x,t] = lsim(sysKF,[u; y'],t);

plot(t,xtrue,'-',t,x,'--','LineWidth',2)

figure
plot(t,y)
hold on
plot(t,xtrue(:,1),'r')
plot(t,x(:,1),'k--')
```

|         |         |
|---------|:--------|
|![graph](./images/cartpendxpos.png)|<p> <a name="simnoise"> Fig. 5 </a> Simulation of a noise position from an impulse system. Estimate position "cover" the true</p>|
|![graph-est](./images/true-estimate.png)|<p> <a name="true-estimate"> Fig. 6 </a> Showing the state variable "true" and estimate</p>|